# Transhipment: maximum flow

## Introduction to optimization and operations research

Michel Bierlaire


In [ ]:

import sys

import numpy as np
from matplotlib import pyplot as plt
from networkx import (
    DiGraph,
    draw_networkx_nodes,
    draw_networkx_edges,
    draw_networkx_edge_labels,
    Graph,
    draw_networkx_labels,
)
from teaching_optimization.simplex_tableau import SimplexAlgorithmTableau
from teaching_optimization.tableau import SimplexTableau


In this lab, you will model and solve a **maximum flow** task as a special case of the
transhipment framework and interpret the result on a real‑world‑style network. You will
build the network with arc **capacities**, add a **dummy arc** to turn the objective into
a circulation, write the associated **linear optimization problem**, and solve it using the
simplex tableau implementation. Then you will verify optimality by checking flows on
**saturated cuts** and comparing alternative optimal solutions. The goal is to connect the
graph picture (nodes, arcs, cuts) with the algebra of a linear optimization problem and to
understand why maximum flow can be expressed and solved within the transhipment model.

For security
reasons, the city council of Lausanne wants to know how many persons can get
from the train station to the bar ``Great Escape'' during one hour.  The possible links which can be used and their
corresponding capacity are the following:

- Station to Metro: 1500 persons/hour.
- Station to Place de l'Europe: 2700 persons/hour.
- Metro to the Great Escape: 2500 persons/hour.
- Place de l'Europe to Metro: 1800 persons/hour.
- Place de l'Europe to Great Escape: 2200 persons/hour.

We model this as a maximum flow problem.

# Question 1
Code and draw the corresponding network. The network must have the cost, the upper
and lower bounds on each arc, including the additional dummy arc used for counting the flow.

In [ ]:

the_network = DiGraph()


Add nodes

In [ ]:
the_network.add_node('Station')
the_network.add_node('Metro M2')
the_network.add_node('Pl. Europe')
the_network.add_node('Great Escape')


Add arcs with cost, and capacity (lower and upper bound). Here is how to code the first arc.

In [ ]:
the_network.add_edge('Station', 'Metro M2', cost=0, lower_bound=0, upper_bound=1500)

Add the other arcs

In [ ]:
 ...














We save the list of arcs from the original network, to draw them differently.

In [ ]:
original_arcs = list(the_network.edges())


Add the dummy arc.

In [ ]:

 ...






Define coordinates for the plot.

In [ ]:
pos = {
    'Station': (0, 4),
    'Metro M2': (5, 4),
    'Pl. Europe': (3, 0),
    'Great Escape': (8, 0),
}



Function to plot the network.

In [ ]:
def plot_network(network: Graph) -> None:
    """Plot the network and its data.

    :param network: network to plot.
    """

    # Figure size
    plt.figure(figsize=(10, 10))

    # Draw the nodes
    draw_networkx_nodes(network, pos, node_size=1000, node_color='lightblue', alpha=0.5)

    # Move the labels
    shifted_positions = {
        node: (coord[0], coord[1] - 0.2) for node, coord in pos.items()
    }

    # Draw the node labels
    draw_networkx_labels(network, shifted_positions, font_size=12, font_weight='bold')

    # Draw the arcs with labels
    edge_labels = {}
    for u, v, data in network.edges(data=True):
        lower_bound = data['lower_bound']
        upper_bound = data['upper_bound']
        label = f'({lower_bound},{upper_bound})'
        edge_labels[(u, v)] = label

    draw_networkx_edges(
        network, pos, edgelist=original_arcs, arrowstyle='->', arrowsize=20
    )
    # We draw the dummy arc differently.
    dummy_arc = ('Great Escape', 'Station')
    draw_networkx_edges(
        network,
        pos,
        edgelist=[dummy_arc],
        style='dotted',
        arrowstyle='->',
        arrowsize=20,
    )
    draw_networkx_edge_labels(
        network, pos, edge_labels=edge_labels, font_size=10, label_pos=0.3
    )

    # Display the graph
    plt.title('Maximum flow problem')
    plt.axis('off')
    plt.show()


plot_network(network=the_network)



# Question 2
Write down the corresponding optimization problem. Remember that
the max flow is a special case of the transhipment problem.

# Question 3
Solve the problem with the simplex algorithm
A solution to this problem is the following flows:

- $x_{S, M2} = 1500$,
- $x_{S, E} = 2700$,
- $x_{E, M2} = 500$,
- $x_{E, GE} = 2200$,
- $x_{M2, GE} = 2000$.

Another solution is

- $x_{S, M2} = 1500$,
- $x_{S, E} = 2700$,
- $x_{E, M2} = 1000$,
- $x_{E, GE} = 1700$,
- $x_{M2, GE} = 2500$.

Check that they both have the same value for the objective function.

As the problem is solved by the simplex algorithm, no need to transform the network to obtain the standard form.
Simply add slack variables like for general linear optimization problems, even if the matrix of the standard form
problem is not the incidence matrix of a transhipment problem.

In [ ]:













matrix = ...












print(matrix)



The cost vector $c$.

In [ ]:


vector_c = ...



The right-hand-side $b$.

In [ ]:


vector_b = ...


We create the algorithm

In [ ]:
the_algorithm = SimplexAlgorithmTableau(
    objective=vector_c,
    constraint_matrix=matrix,
    right_hand_side=vector_b,
)


We solve the problem

In [ ]:
optimal_tableau: SimplexTableau = the_algorithm.solve()


Check if the problem is feasible

In [ ]:
if optimal_tableau is None:
    print(f'Optimization problem is infeasible.')
    sys.exit()


Optimal solution

In [ ]:
print(optimal_tableau.feasible_basic_solution)


Interpretation in terms of arc flows:

In [ ]:
flow_ge_s = ...
print(f'GE -> S: {flow_ge_s}')

flow_s_m2 = ...
print(f'S -> M2: {flow_s_m2}')

flow_s_e = ...
print(f'S -> E : {flow_s_e}')

flow_e_m2 = ...
print(f'E ->M2 : {flow_e_m2}')

flow_e_ge = ...
print(f'E ->GE : {flow_e_ge}')

flow_m2_ge = ...
print(f'M2 ->GE: {flow_m2_ge}')



Optimal value

In [ ]:
print(f'{optimal_tableau.value_objective_function}')


What is the total flow through the network?

In [ ]:

total_flow = ...
print(f'Total flow through the network: {total_flow}')



# Question 4
Identify a saturated cut in the original graph, that does not involve
the dummy arc.